<a href="https://colab.research.google.com/github/mylesdgarveyphd/Academia-Portfolio/blob/main/neural_network_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generate the Training Data

In [2]:
import os
import random
from PIL import Image, ImageDraw

# Ensure necessary library is installed
def setup_environment():
    try:
        from PIL import Image
    except ImportError:
        import subprocess
        subprocess.check_call(["pip", "install", "pillow"])

setup_environment()

def create_image_with_two(output_dir, image_size=(128, 128), num_images=100):
    """
    Generate images with the concept of "2" embedded in them.

    Args:
        output_dir (str): Directory to save the images.
        image_size (tuple): Size of the generated images (width, height).
        num_images (int): Number of images to generate.
    """
    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_images):
        img = Image.new("RGB", image_size, "white")
        draw = ImageDraw.Draw(img)

        # Randomly decide what kind of "2" representation to draw
        representation_type = random.choice(["lines", "dots", "shapes"])

        if representation_type == "lines":
            # Draw two random lines
            for _ in range(2):
                x1, y1 = random.randint(0, image_size[0]), random.randint(0, image_size[1])
                x2, y2 = random.randint(0, image_size[0]), random.randint(0, image_size[1])
                draw.line((x1, y1, x2, y2), fill="black", width=2)

        elif representation_type == "dots":
            # Draw two random dots
            for _ in range(2):
                x, y = random.randint(0, image_size[0] - 10), random.randint(0, image_size[1] - 10)
                radius = random.randint(3, 10)
                draw.ellipse((x, y, x + radius, y + radius), fill="black")

        elif representation_type == "shapes":
            # Draw two random shapes (rectangles, ovals)
            for _ in range(2):
                x1, y1 = random.randint(0, image_size[0] - 20), random.randint(0, image_size[1] - 20)
                x2, y2 = x1 + random.randint(10, 30), y1 + random.randint(10, 30)
                shape_type = random.choice(["rectangle", "oval"])

                if shape_type == "rectangle":
                    draw.rectangle((x1, y1, x2, y2), outline="black", width=2)
                elif shape_type == "oval":
                    draw.ellipse((x1, y1, x2, y2), outline="black", width=2)

        # Save the image
        img.save(os.path.join(output_dir, f"image_{i:04d}.png"))

# Example usage
output_directory = "generated_images"
create_image_with_two(output_directory, image_size=(128, 128), num_images=500)


#Train the Neural Network

In [ ]:
import os
import random
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim

# Ensure necessary libraries are installed
def setup_environment():
    try:
        import torch
    except ImportError:
        import subprocess
        subprocess.check_call(["pip", "install", "torch"])
setup_environment()

class DynamicANN(nn.Module):
    def __init__(self, input_size, layer_config, output_size=None):
        super(DynamicANN, self).__init__()

        output_size = output_size or input_size

        layers = []
        current_size = input_size

        for neurons in layer_config:
            layers.append(nn.Linear(current_size, neurons))
            layers.append(nn.ReLU())
            current_size = neurons

        layers.append(nn.Linear(current_size, output_size))

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Load and preprocess images
def load_and_preprocess_image(image_path, image_size=(128, 128)):
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    img = img.resize(image_size)
    img_array = np.array(img, dtype=np.float32) / 255.0
    return img_array.flatten()  # Flatten for input into ANN

# Save the output image
def save_output_image(output_tensor, output_path, image_size=(128, 128)):
    output_array = output_tensor.detach().numpy().reshape(image_size) * 255.0
    output_image = Image.fromarray(output_array.astype(np.uint8))
    output_image.save(output_path)

# Train the model
def train_model(output_dir, layer_config=(3, 2, 6), num_iterations=50000):
    image_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.png')]
    input_size = 128 * 128
    output_size = input_size

    # Initialize the neural network with dynamic configuration
    model = DynamicANN(input_size, layer_config)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.00001)

    param_changes = []

    for iteration in range(num_iterations):
        # Randomly select two images
        im1_path, im2_path = random.sample(image_files, 2)
        im1 = load_and_preprocess_image(im1_path)
        im2 = load_and_preprocess_image(im2_path)

        # Convert to tensors
        input_tensor = torch.tensor(im1, dtype=torch.float32).unsqueeze(0)
        target_tensor = torch.tensor(im2, dtype=torch.float32).unsqueeze(0)

        # Forward pass
        optimizer.zero_grad()
        output = model(input_tensor)

        # Compute loss and perform backpropagation
        loss = criterion(output, target_tensor)
        loss.backward()
        optimizer.step()

        # Track parameter changes
        param_change = sum(torch.sum(torch.abs(p.grad)).item() for p in model.parameters() if p.grad is not None)
        param_changes.append(param_change)

        # Print and save output image for every 1000th iteration
        if (iteration + 1) % 1000 == 0:
            layer_config_str = '_'.join(map(str, layer_config))
            output_filename = f"out_iter{iteration + 1}_{layer_config_str}.png"
            save_output_image(output, os.path.join(output_dir, output_filename))
            print(f"Iteration {iteration + 1}/{num_iterations}, Loss: {loss.item():.6f}, Param Change: {param_change:.6f}")
            print(f"Output image saved as {output_filename}")

    # Save final output image
    example_input_path = random.choice(image_files)
    example_input = load_and_preprocess_image(example_input_path)
    example_tensor = torch.tensor(example_input, dtype=torch.float32).unsqueeze(0)
    example_output = model(example_tensor)
    layer_config_str = '_'.join(map(str, layer_config))
    final_output_filename = f"out_final_{layer_config_str}.png"
    save_output_image(example_output, os.path.join(output_dir, final_output_filename))
    print(f"Final output image saved as {final_output_filename}")

    return param_changes

# Example usage
output_directory = "generated_images"
# You can now easily configure the layer structure
param_changes = train_model(output_directory, layer_config=(128,128), num_iterations=50000)

Iteration 1000/50000, Loss: 0.094654, Param Change: 194.713219
Output image saved as out_iter1000_128_128.png
Iteration 2000/50000, Loss: 0.111953, Param Change: 681.736643
Output image saved as out_iter2000_128_128.png


KeyboardInterrupt: 

#Combine visual data into a video

In [ ]:
import cv2
import os

def create_video_from_images(image_dir, output_video_path, frame_rate=5, image_size=(128, 128)):
    """
    Create a video from a sequence of images in a directory.

    Args:
        image_dir (str): Path to the directory containing images.
        output_video_path (str): Path to save the generated video.
        frame_rate (int): Frame rate for the video.
        image_size (tuple): Size of each image in the video.
    """
    def extract_iteration_number(filename):
        try:
            # Extract number after 'iter' and before '_'
            return int(filename.split('_iter')[1].split('_')[0])
        except (IndexError, ValueError):
            return float('inf')  # Non-conforming files go to the end

    # Get and sort image files
    image_files = sorted(
        [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('3_2_6.png')],
        key=lambda x: extract_iteration_number(x)
    )

    if not image_files:
        print("No images found in the directory.")
        return

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for .mp4 files
    video_writer = cv2.VideoWriter(output_video_path, fourcc, frame_rate, image_size)

    for image_file in image_files:
        # Read the image
        img = cv2.imread(image_file)
        # Resize the image to match the video size
        img_resized = cv2.resize(img, image_size)
        # Write the frame to the video
        video_writer.write(img_resized)

    # Release the VideoWriter
    video_writer.release()
    print(f"Video saved to {output_video_path}")

# Example usage
image_directory = "generated_images"  # Directory with your images
video_path = "training_output_video_n300.mp4"  # Path for saving the video
create_video_from_images(image_directory, video_path)


Video saved to training_output_video_n300.mp4


#Sandbox

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

class FastImageDataset(Dataset):
    def __init__(self, image_dir):
        self.images = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png')]
        self.transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((128, 128)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = self.transform(Image.open(img_path))
        return img.view(-1)

class FastImageGPT(nn.Module):
    def __init__(self, input_size, layer_config=(128, 64, 32), num_heads=4):
        super().__init__()
        self.pos_embed = nn.Parameter(torch.randn(1, input_size))

        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads),
            num_layers=len(layer_config)
        )

        self.mlp = nn.Sequential(
            nn.Linear(input_size, layer_config[0]),
            nn.ReLU(),
            nn.Linear(layer_config[0], input_size)
        )

    def forward(self, x):
        x = x + self.pos_embed
        x = x.unsqueeze(0)  # Add batch dimension
        x = self.transformer(x)
        return self.mlp(x.squeeze(0))

def train_model(output_dir, layer_config=(128, 64, 32), num_epochs=50):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    dataset = FastImageDataset(output_dir)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=os.cpu_count())

    model = FastImageGPT(input_size=128*128, layer_config=layer_config).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            batch = batch.to(device)

            optimizer.zero_grad(set_to_none=True)
            output = model(batch)
            loss = criterion(output, batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch}, Avg Loss: {total_loss/len(dataloader):.4f}")

    return model

# Usage
output_directory = "generated_images"
model = train_model(output_directory)